# Command Line Ontology Examples

This demonstrates the `p3` (PhenoPacketPython) command line utility for interfacing with OAK to
do ontology operations.

First let's just see the operations:

In [5]:
!p3 --help

Usage: p3 [OPTIONS] COMMAND [ARGS]...

  Run the CLI.

Options:
  -v, --verbose
  -q, --quiet TEXT
  --help            Show this message and exit.

Commands:
  compare            Rewire all OntologyClass objects to update labels.
  list-terms         Rewire all OntologyClass objects to update labels.
  migrate-obsoletes  Rewire all OntologyClass objects that have an...
  normalize-curies   Rewire all OntologyClass objects to update labels.
  update-labels      Rewire all OntologyClass objects to update labels.
  validate           Validate a collection of phenopackets.
  viz                Rewire all OntologyClass objects to update labels.


## List all terms

This iterates through a whole object showing all OntologyClass objects.

(note: all commands accept any of YAML, JSON, or RDF)

In [6]:
!p3 list-terms -O yaml ../examples/Phenopacket-covid.yaml | head -20

- id: MONDO:0005015
  label: diabetes mellitus
- id: MONDO:0004994
  label: cardiomyopathy
- id: MONDO:0100096
  label: COVID-19
- id: LOINC:26474-7
  label: Lymphocytes [#/volume] in Blood
- id: NCIT:C67245
  label: Thousand Cells
- id: LOINC:26474-7
  label: Lymphocytes [#/volume] in Blood
- id: NCIT:C67245
  label: Thousand Cells
- id: NCIT:C80473
  label: Left Ventricular Assist Device
- id: NCIT:C722
  label: Oxygen
- id: NCIT:C67388
  label: Liter per Minute


## Repair Term Objects

Ontologies are not static:

- terms may become obsolete; in some cases a replacement will be indicated
- labels may change

In the first case, the `migrate-obsoletes` command will auto-replace obsoletes that have a replaced-by

In the second, `update-labels` will modify any labels.

For these commands you can either pass in an ontology object using `--ontology`, **OR** let OAK figure
out how to access the ontology. The default method is sqlite. There may be an initial lag.

For this example, we will use a fake example with a bad label and an obsolete class.

To demonstrate it is bad:

In [7]:
!p3 validate ../tests/input/Phenopacket-migrate-example.yaml

## Validating ../tests/input/Phenopacket-migrate-example.yaml
ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
## ERROR: ('56844-4', 'CURIE')
ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
## ERROR: ('56844-4', 'CURIE')
ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
## ERROR: ('HP:0100637', 'obsolete')
Errors: 3
Traceback (most recent call last):
  File "/Users/cjm/Library/Caches/pypoetry/virtualenvs/phenopackets-JmUWBwH2-py3.9/bin/p3", line 5, in <module>
    main()
  File "/Users/cjm/Library/Caches/pypoetry/virtualenvs/phenopackets-JmUWBwH2-py3.9/lib/python3.9/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/Users/cjm/Library/Caches/pypoetry/virtualenvs/phenopackets-JmUWBwH2-py3.9/lib/python3.9/site-packages/click/core.p

**NOTE** currently OAK doesn't know how to access some ontologies/vocabularies by default, this will be fixed.

**TODO** make this output friendlier

In [9]:
!p3 update-labels ../tests/input/Phenopacket-migrate-example.yaml -o /tmp/fixed.yaml

ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
## Num results: 2
## RESULT: ('LABEL TO BE REPLACED', 'Retinal detachment')
## RESULT: ('This is not an actual phenotype of the disease - present to test obsoletion replacement', 'obsolete Neoplasia of the nose')


We can see the changes that were made here:

In [12]:
!diff -u ../tests/input/Phenopacket-migrate-example.yaml /tmp/fixed.yaml

--- ../tests/input/Phenopacket-migrate-example.yaml	2023-01-18 09:45:47.000000000 -0800
+++ /tmp/fixed.yaml	2023-01-18 18:15:26.000000000 -0800
@@ -303,10 +303,10 @@
       iso8601duration: P6M
   type:
     id: HP:0000541
-    label: LABEL TO BE REPLACED ## Retinal detachment
+    label: Retinal detachment
 - type:
     id: HP:0100637
-    label: This is not an actual phenotype of the disease - present to test obsoletion replacement
+    label: obsolete Neoplasia of the nose
 subject:
   id: proband A
   karyotypicSex: XX


Note in the above the fake label for HP:0100637 was replaced by the current label *obsolete Neoplasia of the nose* but this indicates another issue - a class has been obsoleted since the phenopacket was made!

We can also apply the `migrate-obsoletes` command that will use term-replaced-by annotations:

In [13]:
!p3 migrate-obsoletes ../tests/input/Phenopacket-migrate-example.yaml -o /tmp/fixed.yaml

ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
ERROR:phenopackets.utilities.ontology_utilities:OntologyClass.id must be a CURIE: 56844-4
## Num results: 1
## RESULT: ('HP:0100637', 'HP:0012720')


In [14]:
!diff -u ../tests/input/Phenopacket-migrate-example.yaml /tmp/fixed.yaml

--- ../tests/input/Phenopacket-migrate-example.yaml	2023-01-18 09:45:47.000000000 -0800
+++ /tmp/fixed.yaml	2023-01-18 18:16:51.000000000 -0800
@@ -303,10 +303,10 @@
       iso8601duration: P6M
   type:
     id: HP:0000541
-    label: LABEL TO BE REPLACED ## Retinal detachment
+    label: LABEL TO BE REPLACED
 - type:
-    id: HP:0100637
-    label: This is not an actual phenotype of the disease - present to test obsoletion replacement
+    id: HP:0012720
+    label: Neoplasm of the nose
 subject:
   id: proband A
   karyotypicSex: XX


## Visualizing Ontology terms

We can use OAK viz to visualize all terms (typically phenotypes or disease but could be any)

In [16]:
!p3 viz --ontology sqlite:obo:mondo ../examples/Phenopacket-covid.yaml -o output/covid.png

![img](output/covid.png)